In [1]:
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Categorical

In [15]:
class Actor_critic(nn.Module):
    def __init__(self, in_dim, out_dim):
        super(Actor_critic, self).__init__()
        
        self.critic_net = nn.Sequential(
            nn.Linear(in_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )
        
        self.actor_net = nn.Sequential(
            nn.Linear(in_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, out_dim),
            # // TODO
            nn.Softmax(dim=1)
        )
    
    def forward(self, x):
        value = self.critic_net(x)
        probs = self.actor_net(x)
        
        return Categorical(probs), value

In [ ]:
class Memory:
    def __init__(self):
        self.states = []
        self.actions = []
        self.rewards = []
        self.masks = []
        self.log_probs = []
        self.values = []
    
    def sample(self):
        pass
    
    def __len__(self):
        return len(self.states)
    
    def reset(self):
        

In [16]:
def smooth_plot(factor, item, plot_decay):
    item_x = np.arange(len(item))
    item_smooth = [np.mean(item[i:i+factor]) if i > factor else np.mean(item[0:i+1])
                  for i in range(len(item))]
    for i in range(len(item)// plot_decay):
        item_x = item_x[::2]
        item_smooth = item_smooth[::2]
    return item_x, item_smooth
    
def plot(episode, rewards, losses):
    clear_output(True)
    rewards_x, rewards_smooth = smooth_plot(10, rewards, 500)
    losses_x, losses_smooth = smooth_plot(10, losses, 100000)
    
    plt.figure(figsize=(18, 10))
    plt.subplot(211)
    plt.title('episode %s. reward: %s'%(episode, rewards_smooth[-1]))
    plt.plot(rewards, label="Rewards", color='lightsteelblue', linewidth='1')
    plt.plot(rewards_x, rewards_smooth, label='Smothed_Rewards', color='darkorange', linewidth='3')
    plt.legend(loc='best')
    
    plt.subplot(212)
    plt.title('Losses')
    plt.plot(losses,label="Losses",color='lightsteelblue',linewidth='1')
    plt.plot(losses_x, losses_smooth, 
             label="Smoothed_Losses",color='darkorange',linewidth='3')
    plt.legend(loc='best')
    
    plt.show()

In [17]:
## hyperparameters

env_name = "CartPole-v1"
max_epoch = 10000
ppo_epochs = 3
mini_batch_size = 256
gamma = 0.98
lambda1 = 0.95
learning_rate = 0.001
num_timesteps = 128

## hyperparameters

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

env = gym.make(env_name)
in_dim = env.observation_space.shape[0]
out_dim = env.env.action_space.n
network = Actor_critic(in_dim, out_dim).to(device)
old_network = Actor_critic(in_dim, out_dim).to(device)
old_network.load_state_dict(network.state_dict())
optimizer = optim.Adam(network.parameters(), lr=learning_rate)

In [ ]:
all_rewards = []
all_loss = []

for epoch in range(max_epoch):
    
    state = env.reset()
    
    for _ in range(num_timesteps):
        state = torch.FloatTensor(state).to(device)
        dist, value = old_network(state)
        action = dist.sample()
        next_state, reward, done, _ = env.step(action.cpu().numpy())
        
        states.append(state)
        